In [1]:
from datasets import load_dataset
DATASET_NAME='arc_easy'
dataset_easy = load_dataset("allenai/ai2_arc", "ARC-Easy", split=["train", "test",'validation'])

In [2]:
def get_prompt(json_line, has_choice=False):
    try:
        question = json_line["question"]
        choices = json_line["choices"]
        choice_texts = choices["text"]
        perms=list(range(len(choice_texts)))
        choice_texts = [choice_texts[perms[i]] for i in range(len(choice_texts))]
        candidates = " ".join(
            [
                f"({label if has_choice else ' '}) {text}"
                for text, label in zip(choice_texts, choices["label"])
            ]
        ).replace("\n", " ")
        answer_key = json_line["answerKey"][0]
        answer_key_idx = ord(answer_key) - (ord("A") if answer_key in 'ABCDE' else ord("1"))
        answer_text = choices["text"][answer_key_idx]
        fact=f"{json_line['fact1']}. " if 'fact1' in json_line else ''
        prompt = f"{fact}{question} \\n {candidates}"
        return prompt, answer_text
    except:
        print(answer_key)

In [3]:
from tqdm import tqdm
import pickle
container_train=[]
container_test=[]
for dataset in [dataset_easy]:
    container_train.extend([get_prompt(x, has_choice=False) for x in tqdm(dataset[0])])
    container_test.extend([get_prompt(x, has_choice=False) for x in tqdm(dataset[1])])
pickle.dump(container_train, open(f"{DATASET_NAME}_train.pkl", "wb"))
pickle.dump(container_test, open(f"{DATASET_NAME}_test.pkl", "wb"))

100%|██████████| 2376/2376 [00:00<00:00, 15772.13it/s]
